## Capstone Project
-------

### Stage 2 - Modelling phase
------

#### Importing packages and data
------

Possible packages that need to be installed:

1. SpaCy

<code> conda install -c spacy spacy </code>

2. 'en_core_web_md' - library used in SpaCy

<code> python -m spacy download en_core_web_md </code>

3. WordCloud

<code> conda install -c conda-forge wordcloud </code>

4. Hyperas

<code> conda install -c jaikumarm hyperas </code>

------

In [ ]:
# import packages
# Hyperas/TensorFlow
# the __future__ import command must be in the beginning of the notebook
from __future__ import print_function

from hyperopt import Trials, STATUS_OK, tpe
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform

import tensorflow as tf
from tensorflow.keras.layers import LSTM, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping

# Basics
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import random

# Graphs
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

# Filter warnings
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Preprocessing; model selection and evaluation
from sklearn import pipeline, preprocessing
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score

# text handling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Models
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.neural_network import MLPClassifier
import statsmodels.api as sm

# for custom countvectorizer with SpaCy lemmatization
import spacy
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, VectorizerMixin
from sklearn.base import TransformerMixin, BaseEstimator
from scipy.sparse import csr_matrix

# WordCloud
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [ ]:
# import packages
data = pd.read_csv("saved_csv/df.csv")
data.drop(columns = "Unnamed: 0",inplace=True)

df = data.copy()

## Overview
------

Below outlines the questions to be answered with the cleaned data, and proposed methodology for each question.

<b><i><u>Question #1</b></i></u>

**What are some of the qualitative insights for the tech industry to improve MH support for employees?**

In the survey, there is a question asking participants to briefly describe what the industry can do as a whole to improve mental health (MH) support for their employees. The purpose of the analysis is to see whether there are any *underlying themes or patterns* to the responses.

<i><u>Methodology</i></u>

Process the text using *SpaCy Lemmatizer* and *CountVectorizer*, then visualize the patterns using *WordCloud*. Since the font on the WordCloud is based on the frequency of the words appearing in the entire dataset, the more frequently used words will be larger in the WordCloud, suggesting a pattern or theme.

<b><i><u>Question #2</b></i></u>

**What are the factors that affect comfort level in discussing MH at workplace?**

One proposed hypothesis is that improving comfort level in discussing MH at workplace can contribute to improving MH support for employees in the tech industry. A *prescriptive analysis* will be done to determine the factors using various questions pertaining to employer's MH coverage, personal MH status and experience with having MH conversations at the workplace, and overall perceived ratings in the dataset as features.

<i><u>Methodology</i></u>

Determine the statistical significance of each feature and the model as a whole using *Statsmodels Logistic Regression*, then use either *Random Forrest Classifier* or *XGBoost Classifier* to determine feature importance.

<b><i><u>Question #3</b></i></u>

**Can we predict one's comfort level in discussing MH at workplace using participants' qualitative responses of ways to improve MH support?**

A *predictive model* will be built to determine if the qualitative responses are viable predictors of one's comfort level in discussing MH at workplace.

<i><u>Methodology</i></u>

Design a model using a *Recurrent Neural Network (RNN)*.

### Preparing the data
------

#### Independent Variables for Q2
------

Questions will be grouped into one of the following categories:

- Current employer's MH coverage
- Previous employer's MH coverage
- MH status
- Witnessed experience (of discussing MH in the workplace)
- Overall perceived ratings
- Comfort level discussing MH in the workplace

In [ ]:
# putting questions into categories

current_mh_coverage = ["Does your employer provide mental health benefits as part of healthcare coverage?",
               "Do you know the options for mental health care available under your employer-provided health coverage?",
               "Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",
               "Does your employer offer resources to learn more about mental health disorders and options for seeking help?",
               "Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?",
               "If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?"]

previous_mh_coverage = ["Have your previous employers provided mental health benefits?",
                        "Were you aware of the options for mental health care provided by your previous employers?",
                        "Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?",
                        "Did your previous employers provide resources to learn more about mental health disorders and how to seek help?",
                        "Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?"]

mh_status = ["Do you currently have a mental health disorder?",
             "Have you ever been diagnosed with a mental health disorder?",'Anxiety Disorder', 'Mood Disorder', 
             'Psychotic Disorder','Eating Disorder', 'Neurodevelopmental Disorders','Personality Disorder', 
             'Obsessive-Compulsive Disorder','Post-Traumatic Stress Disorder', 'Dissociative Disorder',
             'Substance-Related and Addictive Disorders', 'Other','Adjustment disorder',
             "Have you had a mental health disorder in the past?",
             "Have you ever sought treatment for a mental health disorder from a mental health professional?",
             "Do you have a family history of mental illness?",
             "How willing would you be to share with friends and family that you have a mental illness?",
             "Would you be willing to bring up a physical health issue with a potential employer in an interview?"]

witnessed_exp = ["Have your observations of how another individual who discussed a mental health issue made you less likely to reveal a mental health issue yourself in your current workplace?",
                 "Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?",
                 "Have you observed or experienced supportive or well handled response to a mental health issue in your current or previous workplace?"]

ratings = df.columns[df.columns.str.contains("Overall")]

demographics = ["What is your age?","What is your gender?","What country do you live in?",
                "What US state or territory do you live in?","What is your race?"]

comfort_talking_current = ["Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?",
                           "Have you ever discussed your mental health with your employer?",
                           "Would you feel comfortable discussing a mental health issue with your coworkers?",
                           "Have you ever discussed your mental health with coworkers?",
                           "Have you ever had a coworker discuss their or another coworker's mental health with you?",
                           "Would you feel more comfortable talking to your coworkers about your physical health or your mental health?",
                           "Would you bring up your mental health with a potential employer in an interview?",
                           "Are you openly identified at work as a person with a mental health issue?"]

comfort_talking_previous = ["Would you have felt more comfortable talking to your previous employer about your physical health or your mental health?",
                            "Would you have been willing to discuss your mental health with your direct supervisor(s)?",
                            "Did you ever discuss your mental health with your previous employer?",
                            "Would you have been willing to discuss your mental health with your coworkers at previous employers?",
                            "Did you ever discuss your mental health with a previous coworker(s)?",
                            "Did you ever have a previous coworker discuss their or another coworker's mental health with you?",
                            "Would you bring up your mental health with a potential employer in an interview?",
                            "Are you openly identified at work as a person with a mental health issue?"]

categories = [current_mh_coverage,previous_mh_coverage,mh_status,witnessed_exp,
              ratings,comfort_talking_current,comfort_talking_previous]

Various questions from current employer's MH coverage, MH status, witnessed experience as well as overall perceived ratings will be chosen as indpendent variables. Demographic data such as age, country of residence and race will be included for generaliability of the results.

Dummy variables will be generated from the responses of each question. Responses will be grouped in two general categories (eg. Yes/No) and for the independent variables, one group of responses will be dropped to avoid multicollinearity.

In [ ]:
# Creating functions for dummy generation
def make_dummies(question,columns_to_keep = "Yes"):
    '''
    The function creates dummy variables for independent variables.
    You can specify the column to KEEP.
    
    Inputs:
    ------
    question: the question/column name that you wish to create a dummy vairable for
    columns_to_keep: a string indicating the answer you want to keep
    
    Returns:
    -------
    result: a DataFrame of the desire column in dummy variables.
    '''
    
    dummies = pd.get_dummies(df_2.loc[:,question])
    for j in range(len(dummies.columns)):
        name = question + "__" + columns_to_keep
        dummies.rename(columns = {columns_to_keep : name},inplace=True)

    result = dummies.loc[:,[name]]
    
    return result

def make_dummies_iloc(question,list_to_keep):
    '''
    The function creates dummy variables for independent variables.
    This is designed for instances when you need to keep multiple responses.
    
    Input:
    ------
    question: the question/column name that you wish to create a dummy variable for
    list_to_keep: a list of numbers corresponding the column numbers of the responses 
                  (after it has been converted to dummy variables) that you want to keep
    
    Returns:
    -------
    result: A DataFrame of the desire column in dummy variables.
    
    '''
    
    dummies = pd.get_dummies(df_2.loc[:,question])
    for j in range(len(dummies.columns)):
        name = question + "__" + dummies.columns[j]
        dummies.rename(columns = {dummies.columns[j] : name},inplace=True)
    
    result = pd.DataFrame()
    for num in list_to_keep:
        to_keep = dummies.iloc[:,num]
        result = pd.concat([result,to_keep],axis=1)
    
    return result

def make_dummies_q(column_num,column_name):
    '''
    This function creates a dummy variable for a particular column,
    and drops a column to avoid multicollinearity since the responses will be included in the independent variables.
    
    Inputs:
    ------
    column_num = an integer of the number of column/question you wish to create a dummy variable for
    column_name = the response that you wish to drop to avoid multicollinearity
    
    Returns:
    -------
    result = a DataFrame of dummy variables, containing n-1 answers
    '''

    result = pd.get_dummies(df_2.iloc[:,column_num])
    result.drop(columns = column_name, inplace=True)
    
    return result

In [5]:
# making a copy of the data
df_2 = df.copy()

In [6]:
# set up 2 dataframes for concatenating data
omitted = pd.DataFrame(columns = ["Question","Answer"])
final = pd.DataFrame()

In [7]:
# current_mh_coverage
for i in [0,2,3]:
    result = make_dummies(current_mh_coverage[i],"Yes")
    final = pd.concat([final,result],axis = 1)

result = make_dummies_iloc(current_mh_coverage[5],[2,-2,-1])
final = pd.concat([final,result],axis = 1)

In [8]:
# witnessed_exp
result = make_dummies(witnessed_exp[0],"Yes")
final = pd.concat([final,result],axis = 1)

In [9]:
# mh_status

# Modifying some responses for ease of sorting
old_answer_1 = "Possibly"
old_answer_2 = "-1"
answer = "Don't Know"

to_dummy = []
for i in [0,1,-5,-3,-1]:
    to_dummy.append(mh_status[i])

for num in [0,2]:
    df_2.loc[:,to_dummy[num]][df_2.loc[:,to_dummy[num]]==old_answer_1]=answer

df_2.loc[:,to_dummy[2]][df_2.loc[:,to_dummy[2]]==old_answer_2]=answer

#creating dummy variables
result = make_dummies(to_dummy[0],"Yes")
final = pd.concat([final,result],axis = 1)

for i in [2,3,4,5,6,7,8,9,10,11,12,13,15,17]:
    final = pd.concat([final,df_2.loc[:,mh_status[i]]],axis = 1)

In [10]:
# ratings
for i in [1,4]:
    final = pd.concat([final,df_2.loc[:,ratings[i]]],axis = 1)

In [11]:
# What is your gender?
gender = make_dummies_q(-7,"Male")

In [12]:
# Grouping countries into 5 continents
country_names = df_2.iloc[:,-6].groupby(df_2.iloc[:,-6]).count().index

df_2["countries_continent"] = df_2.iloc[:,-6]

north_am = [8,34,55]
south_am = [0,6,9,56]
asia = [3,20,23,24,26,28,29,39,43,44,46]
africa = [14,30,33,37,48]
europe = [2,4,5,7,10,11,13,15,16,17,18,19,21,22,25,27,31,32,35,38,40,41,42,45,47,49,50,51,52,53,54]
oceania = [1,36]
did_not_answer = [12]

continents = [north_am,south_am,asia,africa,europe,oceania,did_not_answer]
names = ["North America", "South America", "Asia", "Africa", "Europe", "Oceania", "Did not answer"]

for position,continent in enumerate(continents):
    for num in np.flip(continent):
        df_2.loc[:,"countries_continent"][df_2.loc[:,"countries_continent"]== country_names[num]]=names[position]
        
countries = make_dummies_q(-1,"North America")

In [13]:
# What is your race?
races = make_dummies_q(-5,"Did not answer")
for num in range(-4,0):
    races.drop(columns = [races.columns[num]],inplace=True)

In [14]:
# company_size
company_size = make_dummies_q(2,"0")

# select companies with larger size into the independent variables
size = company_size.iloc[:,[3,5]]

In [15]:
# Creating a table of demographics
demographics = pd.concat([gender,countries,races,size,df_2.iloc[:,[-9,1,3,4]]],axis=1)

In [16]:
# Finalizing the table of independent variables with demographics
independent_q1 = pd.concat([final,demographics],axis=1)

#### Dependent Variables for Q2
------

Various questions from questions from comfort level of discussing MH in the workplace will be chosen as the dependent variable.

Dummy variables will be generated from the responses of each question. Clusters of classes will be created afterwards with unsupervised clustering methods using the dummy variables as independent variables. In contrast to the independent variable, all responses can be kept since there is no issue of multicollinearity with dependent variables.

In [17]:
question = "Would you feel comfortable discussing a mental health issue with your coworkers?"

answers = ["Maybe","No","Not Applicable","Yes"]

dep = df_2[question].copy()

for num in range(len(answers)):
    if num != 3:
        dep[dep==answers[num]] = 0 #Hesitant
    else:
        dep[dep==answers[num]] = 1 #Comfortable

In [18]:
question = "Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?"

answers = ["Maybe","No","Not Applicable","Yes"]

dep_2 = df_2[question].copy()

for num in range(len(answers)):
    if num != 3:
        dep_2[dep_2==answers[num]] = 0 #Hesitant
    else:
        dep_2[dep_2==answers[num]] = 1 #Comfortable

## Question 1

### What are some of the qualitative insights for the tech industry to improve MH support for employees?
-----

Qualitative responses from the question "briefly describe what you think the industry as a whole and/or employers could do to improve mental health support for employees" will be processed using [spacy-vectorizers](https://github.com/mpavlovic/spacy-vectorizers), a custom CountVectorizer with SpaCy lemmatization embedded.

In [ ]:
# Using spacy-vectorizers to process texts
# Code source: https://github.com/mpavlovic/spacy-vectorizers

class SpacyPipeInitializer(object):
    def __init__(self, nlp, join_str=" ", batch_size=10000, n_threads=2):
        self.nlp = nlp
        self.join_str = join_str
        self.batch_size = batch_size
        self.n_threads = n_threads
        
class SpacyPipeProcessor(SpacyPipeInitializer):
    def __init__(self, nlp, multi_iters=False, join_str=" ", batch_size=10000, n_threads=2):
        super(SpacyPipeProcessor, self).__init__(nlp, join_str, batch_size, n_threads)
        self.multi_iters = multi_iters
    
    def __call__(self, raw_documents):
        docs_generator = self.nlp.pipe(raw_documents, batch_size=self.batch_size, n_threads=self.n_threads)
        return docs_generator if self.multi_iters == False else list(docs_generator)
    
class SpacyLemmaCountVectorizer(CountVectorizer):
    
    def __init__(self, input='content', encoding='utf-8',
                 decode_error='strict', strip_accents=None,
                 lowercase=True, preprocessor=None, tokenizer=None,
                 stop_words=None, token_pattern=r"(?u)[^\r\n ]+",
                 ngram_range=(1, 1), analyzer='word',
                 max_df=1.0, min_df=1, max_features=None,
                 vocabulary=None, binary=False, dtype=np.int64, 
                 nlp=None, ignore_chars='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~', 
                 join_str=" ", use_pron=False):
        
        super().__init__(input, encoding, decode_error, strip_accents, 
                                                   lowercase, preprocessor, tokenizer,
                                                   stop_words, token_pattern, ngram_range, 
                                                   analyzer, max_df, min_df, max_features,
                                                   vocabulary, binary, dtype)
        self.ignore_chars = ignore_chars
        self.join_str = ' ' # lemmas have to be joined for splitting
        self.use_pron = use_pron
        self.translate_table = dict((ord(char), None) for char in self.ignore_chars)
        
    def lemmatize_from_docs(self, docs):
        for doc in docs:
            lemmas_gen = (token.lemma_.translate(self.translate_table) if self.use_pron or token.lemma_!='-PRON-' else token.lower_.translate(self.translate_table) for token in doc)  # generator expression
            yield self.join_str.join(lemmas_gen) if self.join_str is not None else [lemma for lemma in lemmas_gen]
    
    def build_tokenizer(self):
        return lambda doc: doc.split()
    
    def transform(self, spacy_docs):
        raw_documents = self.lemmatize_from_docs(spacy_docs)
        return super(SpacyLemmaCountVectorizer, self).transform(raw_documents)
    
    def fit_transform(self, spacy_docs, y=None):
        raw_documents = self.lemmatize_from_docs(spacy_docs)
        return super(SpacyLemmaCountVectorizer, self).fit_transform(raw_documents, y)

In [ ]:
# Grabbing the text responses
corpus = df.iloc[:,-9]

# customization stopwords to filter out some words
stopwords = set(STOPWORDS)
stopwords.update(["mental","health","issue","work",
                  "take","hour","tech","industry","people","employee"])

In [ ]:
# CountVectorizer with SpaCy Lemmatization
nlp = spacy.load('en_core_web_md')

spp = SpacyPipeProcessor(nlp, n_threads=1, multi_iters=True)
spacy_docs = spp(corpus);

slcv = SpacyLemmaCountVectorizer(min_df=3,stop_words=stopwords, ngram_range=(1, 3), ignore_chars='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
slcv.fit(spacy_docs)
count_vectors = slcv.transform(spacy_docs); count_vectors

Once the text has been processed, a WordCloud is generated to visualize the result.

In [ ]:
# Pulling out the list of parsed words and put them into a wordcloud
list_of_words = slcv.vocabulary_.keys()
list_of_words = list(list_of_words)
list_of_words.sort()

wordcloud = WordCloud(background_color="white").generate(" ".join(list_of_words))

plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show();

Words with bigger fonts in the WordCloud are ones that occur in the dataset with higher frequency. From the WordCloud above, it seems like words like "open", "support", "talk" are big themes of the responses. This finding supports the need of looking deeper into factos that affect comfort level in discussing MH in the workplace.

## Question #2

### What are the factors that affect comfort level in discussing MH at workplace?
------

#### a) Comfort discussing with coworkers

In [ ]:
# p-value WITH DEMOGRAPHICS using Statsmodel Logistic Regression

X_1 = independent_q1
Y_1 = dep.values.astype(int)

scaler = preprocessing.StandardScaler()
scaler.fit(X_1)
X_transformed_1 = scaler.transform(X_1)

X_transformed_1 = np.hstack([np.ones([X_transformed_1.shape[0],1]), X_transformed_1])

logit = sm.Logit(Y_1, X_transformed_1)
fitted_model_demo = logit.fit_regularized()
fitted_model_demo.summary()

# alpha = 0.05
# all features together are significant
# Columns that are significant: 1,3,4,5,6,13,20,21,30,32,43,44

In [ ]:
# Creating a model to get the feature importances

# filter depreciation warning that is associated with not using 
# the most updated version of numpy and scikit-learnin the vitual environment
warnings.filterwarnings("ignore",category=DeprecationWarning)

ind_q1 = independent_q1.copy()
ind_q1.columns = [np.arange(len(ind_q1.columns))]

X_train, X_test, y_train, y_test = train_test_split(ind_q1,dep.values.astype(int),test_size = 0.2)

estimators = [("normalize", preprocessing.StandardScaler()),
             ("model",LogisticRegression())]

pipe = pipeline.Pipeline(estimators)

param_grid = [{"model":[XGBClassifier()], 
               "normalize": [preprocessing.StandardScaler(), preprocessing.MinMaxScaler(), None],
               "model__max_depth":[1,2,3,4,5],"model__n_estimators":[50,100,150,200],"model__n_jobs":[6]},
              {"model": [RandomForestClassifier()],
               "normalize": [preprocessing.StandardScaler(), preprocessing.MinMaxScaler(), None],
               "model__n_estimators":[100,150,200],"model__n_jobs":[6]}]

grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=6)
fitted_grid_1 = grid.fit(X_train,y_train)

In [ ]:
fitted_grid_1.best_estimator_

In [ ]:
fitted_grid_1.best_score_

In [ ]:
fitted_grid_1.score(X_test,y_test)

In [ ]:
# f1 score of the model
y_pred = fitted_grid_1.predict(X_test)

f1_score(y_test,y_pred)

In [ ]:
# Ranking of factors that are important for predicting comfort level, from most to least important
indices = np.flip(fitted_grid_1.best_estimator_.named_steps["model"].feature_importances_.argsort())

# creating a list of factors that are statistically significant
sig_list = [1,3,4,5,6,13,20,21,30,32,43,44]

numbers = []
for position,idx in enumerate(indices):
    for num in sig_list:
        if idx == num:
            numbers.append(idx)

independent_q1.columns[numbers]

------

#### b) Comfort discussing with supervisors
------

In [ ]:
# 2nd question regarding supervisors
# p-value WITH DEMOGRAPHICS

X_1 = independent_q1
Y_1 = dep_2.values.astype(int)

scaler = preprocessing.StandardScaler()
scaler.fit(X_1)
X_transformed_1 = scaler.transform(X_1)

X_transformed_1 = np.hstack([np.ones([X_transformed_1.shape[0],1]), X_transformed_1])

logit = sm.Logit(Y_1, X_transformed_1)
fitted_model_demo = logit.fit_regularized()
fitted_model_demo.summary()

# alpha = 0.05
# all features together are significant
# Columns that are significant: 3,4,5,9,15,19,20,21,22,23,35

In [ ]:
# Creating a model to get the feature importances

# filter depreciation warning that is associated with not using 
# the most updated version of numpy and scikit-learnin the vitual environment
warnings.filterwarnings("ignore",category=DeprecationWarning)

ind_q1 = independent_q1.copy()
ind_q1.columns = [np.arange(len(ind_q1.columns))]

X_train, X_test, y_train, y_test = train_test_split(ind_q1,dep_2.values.astype(int),test_size = 0.2)

estimators = [("normalize", preprocessing.StandardScaler()),
             ("model",LogisticRegression())]

pipe = pipeline.Pipeline(estimators)

param_grid = [{"model":[XGBClassifier()], 
               "normalize": [preprocessing.StandardScaler(), preprocessing.MinMaxScaler(), None],
               "model__max_depth":[1,2,3,4,5],"model__n_estimators":[50,100,150,200],"model__n_jobs":[6]},
              {"model": [RandomForestClassifier()],
               "normalize": [preprocessing.StandardScaler(), preprocessing.MinMaxScaler(), None],
               "model__n_estimators":[100,150,200],"model__n_jobs":[6]}]

grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=6)
fitted_grid_2 = grid.fit(X_train,y_train)

In [ ]:
fitted_grid_2.best_score_

In [ ]:
fitted_grid_2.score(X_test,y_test)

In [ ]:
y_pred = fitted_grid_2.predict(X_test)

f1_score(y_test,y_pred)

In [ ]:
# Ranking of factors that are important for predicting comfort level, from most to least important
indices = np.flip(fitted_grid_2.best_estimator_.named_steps["model"].feature_importances_.argsort())

# creating a list of factors that are statistically significant
sig_list = [3,4,5,9,15,19,20,21,22,23,35]

numbers = []
for position,idx in enumerate(indices):
    for num in sig_list:
        if idx == num:
            numbers.append(idx)

independent_q1.columns[numbers]

### What can companies to do encourage their employees to seek treatment?
------

In [ ]:
# set up 2 dataframes for concatenating data
omitted = pd.DataFrame(columns = ["Question","Answer"])
final = pd.DataFrame()

# current_mh_coverage
for i in [0,2,3]:
    result = make_dummies(current_mh_coverage[i],"Yes")
    final = pd.concat([final,result],axis = 1)

result = make_dummies_iloc(current_mh_coverage[5],[2,-2,-1])
final = pd.concat([final,result],axis = 1)

In [ ]:
# creating a table of independend variables for this question
ind_variable = pd.concat([final,demographics],axis=1)

In [ ]:
# determining statistical significance using statsmodel Logistic Regression
dep_question = "Have you ever sought treatment for a mental health disorder from a mental health professional?"

X_1 = ind_variable
Y_1 = df_2.loc[:,dep_question]

scaler = preprocessing.StandardScaler()
scaler.fit(X_1)
X_transformed_1 = scaler.transform(X_1)

X_transformed_1 = np.hstack([np.ones([X_transformed_1.shape[0],1]), X_transformed_1])

logit = sm.Logit(Y_1, X_transformed_1)
fitted_model_sp = logit.fit_regularized()
fitted_model_sp.summary()

# alpha = 0.05
# all features together are significant
# Columns that are significant: 0,5,6,7,8,12,14

In [ ]:
# Creating a model to get the feature importance
ind_alt = ind_variable.copy()
ind_alt.columns = [np.arange(len(ind_alt.columns))]

X_train, X_test, y_train, y_test = train_test_split(ind_alt,df_2.loc[:,dep_question],test_size = 0.2)

estimators = [("normalize", preprocessing.StandardScaler()),
             ("model",LogisticRegression())]

pipe = pipeline.Pipeline(estimators)

param_grid = [{"model":[XGBClassifier()], 
               "normalize": [preprocessing.StandardScaler(), preprocessing.MinMaxScaler(), None],
               "model__max_depth":[1,2,3,4,5],"model__n_estimators":[50,100,150,200],"model__n_jobs":[6]},
              {"model": [RandomForestClassifier()],
               "normalize": [preprocessing.StandardScaler(), preprocessing.MinMaxScaler(), None],
               "model__n_estimators":[100,150,200],"model__n_jobs":[6]}]

grid = GridSearchCV(pipe, param_grid, cv=5)
fitted_grid_3 = grid.fit(X_train,y_train)

In [ ]:
fitted_grid_3.best_estimator_

In [ ]:
fitted_grid_3.best_score_

In [ ]:
fitted_grid_3.score(X_test,y_test)

In [ ]:
y_pred = fitted_grid_3.predict(X_test)

f1_score(y_test,y_pred)

In [ ]:
# Ranking of factors that are important for predicting comfort level, from most to least important
indices = np.flip(fitted_grid_3.best_estimator_.named_steps["model"].feature_importances_.argsort())

# displaying features that have statistical significance
sig_list = [0,5,6,7,8,12,14]
numbers = []
for position,idx in enumerate(indices):
    for num in sig_list:
        if idx == num:
            numbers.append(idx)

ind_variable.columns[numbers]

## Question 3

### Can we predict one's comfort level in discussing MH at workplace using participants' qualitative responses of ways to improve MH support?
------

In [19]:
# Grabbing the responses as independent variables
corpus = df.iloc[:,-9]

# Dependent variables
question = "Would you feel comfortable discussing a mental health issue with your coworkers?"

answers = ["Maybe","No","Not Applicable","Yes"]

dep = df[question].copy()

for num in range(len(answers)):
    if num != 3:
        dep[dep==answers[num]] = 0 #Hesitant
    else:
        dep[dep==answers[num]] = 1 #Comfortable

In [20]:
dep.groupby(dep).count()

Would you feel comfortable discussing a mental health issue with your coworkers?
0    855
1    318
Name: Would you feel comfortable discussing a mental health issue with your coworkers?, dtype: int64

In [21]:
# Creating a table with both independent and dependent variables
table = pd.concat([corpus,dep],axis=1)

# dropping columns that did not answer the question
index = table[table.iloc[:,0]=="Did not answer"].index
table.drop(index,axis=0,inplace=True)

# resetting the index
table = table.reset_index()
table.drop("index",axis=1,inplace=True)

In [22]:
# split the dataset into training/test sets
x_train, x_test, y_train, y_test = train_test_split(table.iloc[:,0].values,
                                                    table.iloc[:,1].values,test_size = 0.2)

In [23]:
# Process and transform x_train

# Lemmatization using SpaCy
nlp = spacy.load('en_core_web_md')

# customizing stopwords to exclude certain stopwords
stopwords = set(STOPWORDS)

words = ["against","all","aren't","can't","can","cannot","could","couldn't","did",
         "didn't","doing","don't","hasn't","hadn't","ever","few","mustn't","once","shan't"]

for word in words:
    stopwords.remove(word)

sentences = []

for num in range(len(x_train)):
    doc = nlp(x_train[num])

    sentence = []
    for token in doc:
        sentence.append(token.lemma_)

    sentences.append(" ".join(sentence))

# Processing text with TfidfVectorizer
tf_model = TfidfVectorizer(stop_words=stopwords,ngram_range=(1,3), min_df=3)
tf_vectors = tf_model.fit_transform(sentences); tf_vectors

<631x949 sparse matrix of type '<class 'numpy.float64'>'
	with 9550 stored elements in Compressed Sparse Row format>

In [24]:
from imblearn.over_sampling import SMOTE
from collections import Counter

sm = SMOTE(n_jobs = 6)
X_res,y_res = sm.fit_resample(tf_vectors.toarray(),y_train)
Counter(y_res)

Counter({1: 451, 0: 451})

In [25]:
# Process and transform x_test
sentences = []

for num in range(len(x_test)):
    doc = nlp(x_test[num])

    sentence = []
    for token in doc:
        sentence.append(token.lemma_)

    sentences.append(" ".join(sentence))

x_test_vectors = tf_model.transform(sentences); x_test_vectors

<158x949 sparse matrix of type '<class 'numpy.float64'>'
	with 1924 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

estimators = [("normalize", preprocessing.StandardScaler()),
             ("model",LogisticRegression())]

pipe = pipeline.Pipeline(estimators)

param_grid = [{"model": [RandomForestClassifier()],
               "normalize": [None],"model__n_jobs":[6]},
              {"model": [GradientBoostingClassifier()],"normalize": [None]},
              {"model":[LogisticRegression()],"normalize": [None]}]

grid = GridSearchCV(pipe, param_grid, cv=5)
fitted_grid_3 = grid.fit(X_res,y_res.astype(int))

In [ ]:
fitted_grid_3.best_estimator_

In [ ]:
fitted_grid_3.best_score_

In [ ]:
fitted_grid_3.score(x_test_vectors.toarray(),y_test.astype(int))

In [ ]:
y_pred = fitted_grid_3.predict(x_test_vectors.toarray())

f1_score(y_test.astype(int),y_pred)

In [32]:
from mlxtend.classifier import StackingCVClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

base_models = [SVC(), LogisticRegression(), RandomForestClassifier()]

base_models = [(f'{model.__class__.__name__}-{i}', model) for i, model in enumerate(base_models)]

stacked_model = StackingCVClassifier(classifiers=[model for _, model in base_models],
                                     meta_classifier=GradientBoostingClassifier(), 
                                     use_features_in_secondary=False)

params = {'svc__kernel': ['rbf','poly','sigmoid'],
          'randomforestclassifier__n_estimators': [50,100],'randomforestclassifier__n_jobs': [6], 
          'logisticregression__C': [1e-4,1e-2,1,10],'logisticregression__penalty': ['l1','l2'],
          'logisticregression__n_jobs': [6], 'meta-gradientboostingclassifier__n_estimators' : [50,100],
          'meta-gradientboostingclassifier__max_depth': [3]}

grid = GridSearchCV(estimator=stacked_model, param_grid=params, cv=3,refit=True)
grid.fit(X_res, y_res.astype(int))

GridSearchCV(cv=3, error_score='raise',
       estimator=StackingCVClassifier(classifiers=[SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False), LogisticRegression(C=1.0, clas...           use_clones=True, use_features_in_secondary=False,
           use_probas=False, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'svc__kernel': ['rbf', 'poly', 'sigmoid'], 'randomforestclassifier__n_estimators': [50, 100], 'randomforestclassifier__n_jobs': [6], 'logisticregression__C': [0.0001, 0.01, 1, 10], 'logisticregression__penalty': ['l1', 'l2'], 'logisticregression__n_jobs': [6], 'meta-gradientboostingclassifier__n_estimators': [50, 100], 'meta-gradientboostingclassifier__max_depth': [3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [33]:
grid.best_score_

0.8037694013303769

In [34]:
y_pred = grid.predict(x_test_vectors.toarray())

f1_score(y_test.astype(int),y_pred)

0.1754385964912281

In [ ]:
def NN_model():
    model = Sequential()

    model.add(Dense(64,activation="relu", input_shape = (3,)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(2, activation="softmax"))

    # setting up SGD (optimizer) hyperparameters
    sgd = SGD(lr=0.0001, decay=0.0, momentum = 0.0, nesterov=False, clipnorm=2.0)

    # compile model
    model.compile(loss="sparse_categorical_crossentropy", optimizer = sgd, metrics = ["accuracy"])
    
    return model

In [ ]:
base_models = [KNeighborsClassifier(),LogisticRegression(),
               RandomForestClassifier()]

base_models = [(f'{model.__class__.__name__}-{i}', model) for i, model in enumerate(base_models)]

stacked_model = StackingCVClassifier(classifiers=[model for _, model in base_models],
                                     meta_classifier=KerasClassifier(build_fn=NN_model, batch_size = 16, epochs = 4, validation_split = 0.2), 
                                     use_features_in_secondary=False)

params = {'kneighborsclassifier__n_neighbors': [5,10,15],'kneighborsclassifier__n_jobs': [6],
          'randomforestclassifier__n_estimators': [50,100],'randomforestclassifier__class_weight': ["balanced"],
          'randomforestclassifier__n_jobs': [6], 'logisticregression__C': [1e-4,1e-2,1,10],
          'logisticregression__penalty': ['l1','l2'],'logisticregression__n_jobs': [6]}

grid = GridSearchCV(estimator=stacked_model, param_grid=params, cv=3,refit=True)
grid.fit(X_res, y_res.astype(int))

In [ ]:
grid.best_score_

In [ ]:
y_pred = grid.predict(x_test_vectors.toarray())

f1_score(y_test.astype(int),y_pred)

In [ ]:
# input an response and see if the model predicts correctly
response = input("Briefly describe what you think the tech industry as a whole and/or \
employers could do to improve mental health support for employees.")

print("Processing...")

# Text processing to prepare data for RNN
nlp = spacy.load('en_core_web_md')

sentences = []
doc = nlp(response)

sentence = []
for token in doc:
    sentence.append(token.lemma_)

sentences.append(" ".join(sentence))

print("Almost there...")

# Processing text with TfidfVectorizer
tf_vectors = tf_model.transform(sentences)

# predicting the result using the model
y_pred = grid.predict(tf_vectors.toarray())
y_pred
# # printing the result
# if y_pred[0][0] > 0.5:
#     print("The model predicts you are hesitant with discussing MH issue with your coworkers.")
# else:
#     print("The model predicts you to have comfortable with discussing MH issue with your coworkers.")